The heats are stored using a dictionary. basically everything has an "exposed" end to simulate what a player using the in-game pact will see in the UI.

The index is the level that the condition is raised to. Initialized to an array of zeros.

If a heat's entry gets picked, the index gets advanced by 1. 


We need to make sure that if a pact is used up, the "exposed" number is 0 and cannot be selected. tacked onto the end of all heat costs.

This is for quick reddit pasting.


In [1]:
# !conda install tabulate -y

In [2]:
import numpy as np
import pandas as pd

## stuff for random heats

In [3]:
def heat_check(heats_values,active_index):
    # adds up heat values for a generated combination.
    # just to check that the heat adds up
    heat_total = 0
    for i_ind, ind in enumerate(active_index):
        if ind > 0:
            active_heats = heats_values[i_ind][:ind]
            heat_total += sum(active_heats)         
    return heat_total

In [4]:
def get_random_heats_reddit(heat_thresholds,init_heats=None,restrict_hs=True):

    heats_dict = {

        "Hard Labor": [1,1,1,1,1,0],
        "Lasting Consequences": [1,1,1,1,0],
        "Convenience Fee": [1,1,0],
        "Jury Summons":[1,1,1,0],
        "Extreme Measures":[1,2,3,4,0],
        "Calisthenics Program":[1,1,0],
        "Benefits Package":[2,3,0],
        "Middle Management":[2,0],
        "Underworld Customs":[2,0],
        "Forced Overtime":[3,3,0],
        "Heightened Security":[1,0],
        "Routine Inspection":[2,2,2,2,0],
        "Damage Control":[1,1,0],
        "Approval Process":[2,3,0],
        "Tight Deadline":[1,2,3,0]

    }
    
    # mods to not have runs be absurdly unfun
    heats_dict["Calisthenics Program"] = [0]
    heats_dict["Damage Control"] = [1,0]
    
    output_dict = {}
    
    # hs removed for below 
    # if restrictions placed on hs:
    # 24-32 has hs if it's not cancer 
    # no restrictions on 32.
    
    heats_values = [i for i in heats_dict.values()]
    heats_names = [i for i in heats_dict.keys()]
    
    if init_heats is not None:
        if len(heats_dict) != len(init_heats):
            print("initial heat array wrong length")
            
            return
        
        else:
            active_index = init_heats
            running_heat = heat_check(heats_values,active_index)
        
    else:
        active_index = np.zeros(len(heats_dict),dtype="int8")
        running_heat = 0

    for max_heat in heat_thresholds:
        # can turn this into a function lol
        if max_heat < 24:
            heats_dict["Heightened Security"] = [0]
            heats_dict["Routine Inspection"] = [0]
        elif max_heat in range(24,32) and not restrict_hs:
            heats_dict["Heightened Security"] = [1,0] 
            heats_dict["Routine Inspection"] = [2,0]
        elif max_heat >= 32:
            heats_dict["Heightened Security"] = [1,0]
            heats_dict["Routine Inspection"] = [2,0]
            
        while running_heat < max_heat:

            # generate the outer layer. Bascially the numbers on the right of the pact UI.
            outer_layer = []
            for ih, h in enumerate(heats_dict.values()):
                outer_layer.append(h[active_index[ih]])
            outer_layer = np.asarray(outer_layer)

            # get how much heat we can still add to not exceed the max
            remaining_heat = max_heat - running_heat

            # get the indices of what heats we can still add
            # the layer must not be 0 (ie still have stuff left)
            # and must be LEQ to the remaining heat
            valid_heat_inds = np.where( (outer_layer != 0) & (outer_layer <= remaining_heat) )[0]

            # pick a heat INDEX that you want to raise
            selected_heat_ind = np.random.choice(valid_heat_inds)

            # figure out how much to increment the running heat
            # basically, get the list of heats first, then get the heat level
            running_heat += heats_values[selected_heat_ind][active_index[selected_heat_ind]]

            # update active heat levels
            active_index[selected_heat_ind] += 1
            
        output_dict[f"**{max_heat}**"] = active_index.copy()

    out_df = pd.DataFrame(output_dict,index=heats_names)
    out_df[out_df.eq(0)] = ""
    print(out_df.to_markdown())
#     return output_dict

## weapons and other

In [5]:
def get_random_weapon():
    
    weapons = ["Zag sword","Nem sword","Posi sword","Arthur sword",
               "Zag spear","Hades spear","Achilles spear","Guan Yu spear",
               "Zag shield","Chaos shield","Zeus shield","Beowulf shield",
               "Zag bow","Chiron bow","Hera bow","Rama bow",
               "Zag fists","Talos fists","Demeter fists","Gilga fists",
               "Zag rail","Hestia rail","Eris rail","Lucifer rail"]
    

    selected_weapon = np.random.choice(weapons)
    print("Weapon:",selected_weapon)
    
    return selected_weapon

In [6]:
def get_random_companion():
    companions = ["Battie","Mort","Fidi","Rib","Antos"]
    print("Companion",np.random.choice(companions))

tartarus: anything

Asphodel: anything

Elysium: if tartarus != Asphodel, cannot be tartarus

Styx: if tartarus != Elysium and Asphodel != Elysium, cannot be those 2

keepsake can be KEPT in consecutive regions, but once put down, cannot be picked back up again.


In [7]:
def get_random_keepsakes(special_weight=1):
    
    keepsakes = ['Adamant Arrowhead', 'Black Shawl', 'Blood-Filled Vial', 'Bone Hourglass', 'Broken Spearpoint',
                 'Chthonic Coin Purse', 'Conch Shell', 'Cosmic Egg', 'Distant Memory', 'Eternal Rose', 'Evergreen Acorn',
                 'Frostbitten Horn', 'Harpy Feather Duster', 'Lucky Tooth', 'Myrmidon Bracer','Old Spiked Collar',
                 'Overflowing Cup', 'Owl Pendant', 'Pom Blossom', 'Shattered Shackle', 'Skull Earring', 'Thunder Signet',
                 'Lambent Plume','Pierced Butterfly','Sigil of the Dead']
    
    # lower the probabilities for the last 3 keepsakes 
    # to [special_weight] times lower than the others
    
    # if one wants to weigh the other keepsakes as well, it might be better to write
    # another function that generates the initial weights vector, to spread out the clutter.
    k_weights = np.full(len(keepsakes),special_weight)
    k_weights[-3:] = 1
    k_weights = k_weights/sum(k_weights)
    
    tartarus = np.random.choice(keepsakes,p=k_weights)
    asphodel = np.random.choice(keepsakes,p=k_weights)

    if tartarus != asphodel:
        # zero the weight for the returned keepsake and re-normalize weights
        k_weights[keepsakes.index(tartarus)] = 0
        k_weights = k_weights/sum(k_weights)

    elysium = np.random.choice(keepsakes,p=k_weights)


    if asphodel != elysium:
        # zero and re-normalize again
        k_weights[keepsakes.index(asphodel)] = 0
        k_weights = k_weights/sum(k_weights)

    styx = np.random.choice(keepsakes,p=k_weights)
    
    out_df = pd.DataFrame(data={"**Region**":["Tartarus","Asphodel","Elysium","Styx"],
                                "**Keepsake**":[tartarus,asphodel,elysium,styx]})
    print(out_df.to_markdown(index=False))

In [8]:
def get_random_mirror_reddit():
    mirror = [
        ("Shadow Presence","Fiery Presence"),
        ("Chthonic Vitality","Dark Regeneration"),
        ("Death Defiance","Stubborn Defiance"),
        ("Greater Reflex","Ruthless Reflex"),
        ("Boiling Blood","Abyssal Blood"),
        ("Infernal Soul","Stygian Soul"),
        ("Deep Pockets","Golden Touch"),
        ("Thick Skin","High Confidence"),
        ("Privileged Status","Family Favorite"),
        ("Olympian Favor","Dark Foresight"),
        ("Gods' Pride","Gods' Legacy"),
        ("Fated Authority","Fated Persuasion")
    ]
    mirror_choices = [np.random.choice(i) for i in mirror]
    
    purples = [i[0] for i in mirror]
    greens = [i[1] for i in mirror]
    out_df = pd.DataFrame(data={"**Purple**":purples,"**Green**":greens})
    out_df[~out_df.isin(mirror_choices)] = ""
    
    print(out_df.to_markdown(index=False))
    
    return mirror_choices


## actually generate kit

mirror, weapon and companion are simple random choices

For keepsakes, Lambent Plume, Pierced Butterfly and Sigil of the Dead can be weighted to appear less frequently than the other keepsakes. The variable for get_random_keepsakes lowers the probabilities of those 3 keepsakes by that many times. 

"sane" keepsake stuff to take into consideration can be:

- no shackle in Elysium and Styx
- no Sigil pickup in Styx. Keeping it through is fine. 
- butterfly (and maybe plume), if picked up, must be kept through the whole run.

- new add: no distant memery if weapon is melee. not actually implemented, just manually fix

for "bare" sanity, just no Sigil in Styx. Everything else goes. 
OR just offer alternatives if Sigil is taken.

tbh, we can just NOT take these into consideration, and fix on the fly.

get_random_heats takes in a list of heats, in case one wants to generate heat sets for various game experience levels. Each higher heat contains all pact elements of the lower heats before it. If only 1 heat is needed, just put a 1-element list, like [8]

In [9]:
# a spare cell to generate alternative options if something is too janky
get_random_weapon()

Weapon: Zag rail


'Zag rail'

In [10]:
wpn = get_random_weapon()
print()

melee_wpn =  any([i in wpn for i in ["sword","spear","fists"]])

mirror = get_random_mirror_reddit()
print()

hs_meter = sum([i in mirror for i in ["Stubborn Defiance","High Confidence"]])
hs_meter += ("Ruthless Reflex" in mirror)

restrict_hs = melee_wpn and hs_meter >= 2
print(restrict_hs)

get_random_companion()
print()
get_random_keepsakes(10)
print()

# init_heats = np.array([0,0,0,0,0,0,0,0,0,0,0,1,0,1,0])
init_heats = None
get_random_heats_reddit([4,8,16,24,32],init_heats=init_heats,restrict_hs=restrict_hs)

Weapon: Posi sword

| **Purple**     | **Green**         |
|:---------------|:------------------|
|                | Fiery Presence    |
|                | Dark Regeneration |
|                | Stubborn Defiance |
|                | Ruthless Reflex   |
|                | Abyssal Blood     |
|                | Stygian Soul      |
| Deep Pockets   |                   |
| Thick Skin     |                   |
|                | Family Favorite   |
| Olympian Favor |                   |
| Gods' Pride    |                   |
|                | Fated Persuasion  |

True
Companion Antos

| **Region**   | **Keepsake**     |
|:-------------|:-----------------|
| Tartarus     | Frostbitten Horn |
| Asphodel     | Skull Earring    |
| Elysium      | Lucky Tooth      |
| Styx         | Evergreen Acorn  |

|                      | **4**   | **8**   | **16**   | **24**   | **32**   |
|:---------------------|:--------|:--------|:---------|:---------|:---------|
| Hard Labor           | 1       | 2  

In [11]:
get_random_keepsakes(10)

| **Region**   | **Keepsake**      |
|:-------------|:------------------|
| Tartarus     | Evergreen Acorn   |
| Asphodel     | Cosmic Egg        |
| Elysium      | Bone Hourglass    |
| Styx         | Adamant Arrowhead |


In [12]:
get_random_heats_reddit([4,8,16,24,32],restrict_hs=restrict_hs)

|                      | **4**   | **8**   | **16**   | **24**   | **32**   |
|:---------------------|:--------|:--------|:---------|:---------|:---------|
| Hard Labor           |         |         |          |          | 4        |
| Lasting Consequences | 1       | 1       | 2        | 3        | 4        |
| Convenience Fee      |         |         |          |          |          |
| Jury Summons         |         |         |          | 1        | 1        |
| Extreme Measures     | 1       | 1       | 1        | 1        | 2        |
| Calisthenics Program |         |         |          |          |          |
| Benefits Package     |         | 1       | 1        | 1        | 1        |
| Middle Management    |         |         | 1        | 1        | 1        |
| Underworld Customs   |         |         |          |          |          |
| Forced Overtime      |         |         | 1        | 2        | 2        |
| Heightened Security  |         |         |          |         